In [1]:
import os
import glob
import json
import torch
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict
from tqdm.notebook import tqdm

from modules import Darknet
from coco import (
    load_classes, compute_coco_detections, compute_name_id_mappings,
    map_ids, convert_coco_detections
)
from utils import write_results
from visualize import (
    read_img, resize_aspect, img_to_tensor,
    inv_aspect_transform_results, clamp_results,
    aspect_transform, uniform_color_palette,
    draw_predictions, draw_palette
)
from metrics import compute_interpolated_precision, bbox_metrics, bbox_iou, bbox_match

In [2]:
coco_val_path = '/media/semyon/Data/Documents/coco/val2017'
coco_val_preds_path = '/media/semyon/Data/Documents/coco/preds_pt'

In [3]:
classes = load_classes('coco.names')
num_classes = len(classes)

In [4]:
paths = list(glob.glob(os.path.join(coco_val_path, '*')))

In [5]:
def compute_image_name(img_path):
    img_file_name = os.path.split(img_path)[1]
    img_id = os.path.splitext(img_file_name)[0]
    return img_id

In [6]:
def compute_output_path(img_path, output_dir):
    img_name = compute_image_name(img_path)
    result_file_name = f'{img_name}.pt'
    result_path = os.path.join(output_dir, result_file_name)
    return result_path

In [7]:
annotations_path = '/media/semyon/Data/Documents/coco/annotations/instances_val2017_nocrowd.json'
with open(annotations_path, 'r') as input_file:
    annotations = json.load(input_file)
coco_cat_ids_mapping = '/media/semyon/Data/Documents/coco/coco_cat_ids_mapping.json'
with open(coco_cat_ids_mapping, 'r') as input_file:
    id_mapping = json.load(input_file)
id_mapping = {v: int(k) for k, v in id_mapping.items()}
coco_bboxes = convert_coco_detections(annotations, id_mapping)

In [8]:
n_preds = 100
iou_threshold = 0.5
area_range = [0 ** 2, 1e5 ** 2]
match = defaultdict(list)
for i, path in enumerate(paths):
    image_name = os.path.split(path)[1]
    pred_path = compute_output_path(path, coco_val_preds_path)
    if image_name in coco_bboxes:
        targets = coco_bboxes[image_name]
    else:
        targets = torch.empty(0, 5, dtype=torch.float32)
    preds = torch.load(pred_path)
    if preds.numel() == 0:
        preds = torch.empty(0, 7, dtype=torch.float32)
    for c in torch.unique(targets[:, 4]):
        c = int(c.item()) - 1
        p = preds[preds[:, 6] == c][:, :5]
        t = targets[targets[:, 4] == c + 1][:, :4]
        ious = bbox_iou(p[:, :4], t)
        match[c].append(
            bbox_match(
                p, t, ious,
                iou_threshold=iou_threshold,
                n_preds=n_preds,
                area_range=area_range
            )
        )

In [9]:
num_points = 101
results = np.full((num_classes, num_points), -1, dtype=np.float32)
for c in range(num_classes):
    if c not in match:
        print('no class', c)
        continue
    m = match[c]
    num_targets = sum([rec['num_targets'] for rec in m])
    bbox_m = torch.cat([rec['match'] for rec in m])
    bbox_c = torch.cat([rec['confidence'] for rec in m])

    indices = np.argsort(-bbox_c.detach().cpu().numpy(), kind='mergesort')
    indices = torch.tensor(indices, dtype=torch.int64)
    bbox_c = bbox_c[indices]
    bbox_m = bbox_m[indices]
    
    tp = (bbox_m != -1).cumsum(0)
    fp = (bbox_m == -1).cumsum(0)
    precision = tp / (tp + fp)
    recall = tp / num_targets
    rthresh = torch.linspace(0, 1, num_points)
    iprecision = compute_interpolated_precision(precision, recall, rthresh)
    results[c] = iprecision.detach().cpu().numpy()

In [10]:
results[results > -1].mean()

0.51647633

In [36]:
annotations_path = '/media/semyon/Data/Documents/coco/annotations/instances_val2017.json'
with open(annotations_path, 'r') as input_file:
    annotations = json.load(input_file)
ids = set()
a_nc = []
for a in annotations['annotations']:
    if not a['iscrowd']:
        a_nc.append(a)
annotations['annotations'] = a_nc
annotations_nc_path = '/media/semyon/Data/Documents/coco/annotations/instances_val2017_nocrowd.json'
with open(annotations_nc_path, 'w') as output_file:
    annotations = json.dump(annotations, output_file)